In [10]:
import tensorflow as tf
import matplotlib
from matplotlib import image
import numpy as np

In [6]:
a = np.random.rand(3,4)

In [7]:
a

array([[0.23684173, 0.02858266, 0.49126139, 0.5527044 ],
       [0.35928383, 0.16332037, 0.55413617, 0.15801857],
       [0.77311243, 0.43351726, 0.70342583, 0.65955695]])

In [17]:
np.savetxt('eg.txt',a)

In [18]:
b = np.loadtxt('eg.txt')

In [19]:
b

array([[0.23684173, 0.02858266, 0.49126139, 0.5527044 ],
       [0.35928383, 0.16332037, 0.55413617, 0.15801857],
       [0.77311243, 0.43351726, 0.70342583, 0.65955695]])